In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
INPUT_PATH = '/kaggle/input'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, plot_roc_curve, roc_curve, auc, roc_auc_score
from scipy import interp
from itertools import cycle
import matplotlib.pyplot as plt
import timeit

%matplotlib inline

import gc

In [ ]:
device_name = tf.test.gpu_device_name()
avail_gpu = True
if "GPU" not in device_name:
    avail_gpu = False
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

input_data_path = os.path.join(INPUT_PATH, 'diabetic-retinopathy-resized')
test_data = os.path.join(input_data_path, 'resized_train', 'resized_train')
test_cropped_data = os.path.join(input_data_path, 'resized_train_cropped','resized_train_cropped')
print(test_cropped_data)

In [ ]:
# load pretrained model - deprecated
def load_pretrained(weight='imagenet', include_top=False):
    mobileNetModel = MobileNet(weights=weight, include_top=include_top)
    model = Sequential()
    model.add(mobileNetModel)
    if not include_top:
        model.add(GlobalAveragePooling2D())
        model.add(Dense(5, activation='softmax'))
    model.summary()
    return model

In [ ]:
# functions

def stratified_sampling(data, n_sample=None, stratify=None, random_state=None):
    if not n_sample or not stratify:
        return
    str_sampled_data = data.groupby(stratify, group_keys=False).apply(lambda x: x.sample(int(np.rint(n_sample*len(x)/len(data))), random_state=random_state)).sample(frac=1).reset_index(drop=True)
    return str_sampled_data.reset_index(drop=True)

def data_loader(data, stratify=None, downsampling=False, upsampling=False):
    if stratify == None:
        train, test = train_test_split(data, test_size=0.3)
    else:
        train, test = train_test_split(data, test_size=0.3, stratify=data[stratify])
    if downsampling:
        min_cnt = min(train[stratify].value_counts())
        train = train.groupby(stratify, group_keys=False).apply(lambda x: x.sample(min_cnt)).sample(frac=1).reset_index(drop=True)
    if upsampling:
        max_cnt = max(train[stratify].value_counts())
        train = train.groupby(stratify, group_keys=False).apply(lambda x: x.sample(max_cnt, replace=True)).sample(frac=1).reset_index(drop=True)
    train[['level']].hist(figsize=(15, 15))
    return train, test
    
def micro_f1(y_true, y_pred):
    y_true = np.array([np.argmax(p) for p in y_true]).reshape(-1, 1)
    y_pred = np.array([np.argmax(p) for p in y_pred]).reshape(-1, 1)
    return f1_score(y_true, y_pred, average='micro')

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

def img2vec(path, names, image_size = None):
    vecs = []
    for name in names:
        img = cv2.cvtColor(cv2.imread(os.path.join(path, name+'.jpeg')), cv2.COLOR_BGR2RGB)
        if image_size:
            img = cv2.resize(img, image_size, interpolation = cv2.INTER_CUBIC)
        vecs.append(img)
    return np.array(vecs)

def stratified_sampling(data, n_sample=None, stratify=None, random_state=None):
    if not n_sample or not stratify:
        return
    tmp_data = data.groupby(stratify, group_keys=False).apply(lambda x: x.sample(int(np.rint(n_sample*len(x)/len(data))), random_state=random_state)).sample(frac=1).reset_index(drop=True)
    return tmp_data

def process(df, path = '../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped', image_size=None):
    feature_vec = img2vec(path, df['image'].values, image_size=image_size)
    one_hot = pd.get_dummies(df['level'])
    return feature_vec, one_hot

def model_train(model_loader, x_train, x_label, y_test, y_label, optimizer=None, loss=None, epochs=None, n_layer=0):
    logs= []
    for opt in optimizer:
        for l in loss:
            for e in epochs:
                for i in range(n_layer):
                    model = Sequential()
                    model.add(model_loader(include_top=False))
                    model.add(GlobalAveragePooling2D())
                    model.add(Dense(5, activation='softmax'))
                    for j in range(i+1):
                        model.layers[j].trainable = True
                    print(f'optimizer: {opt}, loss_fn:{l}, epochs: {e}, train_layer: {j}')
                    model.compile(optimizer=opt, loss=l, metrics=['acc'])
                    model.fit(x_train, x_label, epochs=e)
                    prediction = model.predict(y_test)
                    print(f'f1_score: {micro_f1(y_label, prediction)}')
                    plot_multi_roc_curve(y_label, prediction)
                    del model
                    gc.collect()
                    log = f'optimizer: {opt}, loss_fn:{l}, epochs: {e}, train_layer: {j}, micro_f1_score: {micro_f1(y_label, prediction)}'
                    logs.append(log)
    return logs

def plot_multi_roc_curve(y_true, y_pred):
    n_classes = y_true.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(5):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    lw = 5
    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Dataset 준비
labels = pd.read_csv(os.path.join(input_data_path, 'trainLabels_cropped.csv'))
labels = labels.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

# MobileNet Model

## Downsampling

In [ ]:
random_df = stratified_sampling(labels, n_sample=10000, stratify='level', random_state=312)
train, test = data_loader(random_df, stratify='level', downsampling=True)
x_train, x_label = process(train, path=test_cropped_data, image_size = (224, 224))
y_test, y_label = process(test, path=test_cropped_data, image_size = (224, 224))

In [ ]:
model = load_pretrained(include_top=True)
model.summary()

In [ ]:
del model
gc.collect()

## MobileNet은 총 1,000개의 class를 가지고 있는 classifier가지고 있기 때문에 
## 기존의 classifier를 때어주는 작업이 필요하다.```include_top=False```

In [ ]:
model = load_pretrained()

# 기존 weight를 그대로 사용

In [ ]:
no_train_prediction = model.predict(y_test)

In [ ]:
no_train_prediction

In [ ]:
y_label.values

In [ ]:
micro_f1(y_label.values, no_train_prediction)

## Layer Trainable

In [ ]:
optimizers = ['adam']
loss_fn = ['categorical_crossentropy']
epochs = [5, 30]
with tf.device('gpu:0'):
    logs = model_train(MobileNet, x_train, x_label.values, y_test, y_label.values, optimizer=optimizers, loss=loss_fn, epochs=epochs, n_layer=3)

In [ ]:
for log in logs:
    print(log)

# Upsampling

In [ ]:
optimizers = ['adam']
loss_fn = ['categorical_crossentropy']
epochs = [10, 30]
random_df = stratified_sampling(labels, n_sample=2000, stratify='level', random_state=312)
train, test = data_loader(random_df, stratify='level', upsampling=True)
x_train, x_label = process(train, path=test_cropped_data, image_size = (224, 224))
y_test, y_label = process(test, path=test_cropped_data, image_size = (224, 224))

In [ ]:
with tf.device('gpu:0'):
    logs = model_train(MobileNet, x_train, x_label.values, y_test, y_label.values, optimizer=optimizers, loss=loss_fn, epochs=epochs, n_layer=3)

# InceptionV3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

## Downsampling

In [ ]:
random_df = stratified_sampling(labels, n_sample=10000, stratify='level', random_state=312)
train, test = data_loader(random_df, stratify='level', downsampling=True)
x_train, x_label = process(train, path=test_cropped_data, image_size = (299, 299))
y_test, y_label = process(test, path=test_cropped_data, image_size = (299, 299))

In [ ]:
with tf.device('gpu:0'):
    logs = model_train(InceptionV3, x_train, x_label.values, y_test, y_label.values, optimizer=optimizers, loss=loss_fn, epochs=epochs, n_layer=3)

## Upsampling

In [ ]:
random_df = stratified_sampling(labels, n_sample=2000, stratify='level', random_state=312)
train, test = data_loader(random_df, stratify='level', upsampling=True)
x_train, x_label = process(train, path=test_cropped_data, image_size = (299, 299))
y_test, y_label = process(test, path=test_cropped_data, image_size = (299, 299))

In [ ]:
with tf.device('gpu:0'):
    logs = model_train(InceptionV3, x_train, x_label.values, y_test, y_label.values, optimizer=optimizers, loss=loss_fn, epochs=epochs, n_layer=3)

In [ ]:
print(logs)